**Operations for Common Neural Networks**

When training a neural network, we should understand the operations that go into transforming our input into an output. Once we understand these operations (and collections of these operations), it's much easier to grasp the logic of neural networks with (tens or hundreds) of millions (billions) of parameters. We'll also examine all the C/C++ and other code needed to implement these operations in PyTorch. We'll proceed via the PyTorch `examples`, `vision`, `text`, and other libraries of sample network architectures. To save space, we won't write out the full networks. As we go along, we'll express the logic of each architecture in terms of their PyTorch Python implementation (e.g. `nn.Conv2d`).

We'll start with `examples/mnist` and examine the operations inside `main.py`. Since we're defining these outside of our `Net` class, we'll drop the `self.` prefix for each of the ops in the `__init__` constructor. Before we proceed, we'll need to do four imports:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

`torch` is the PyTorch Python package, `torch.nn` enables us to define neural network layers as `Module`s. `torch.nn.functional` contains implementations of a range of operations (including those defined as modules in `torch.nn`. 

We define our first two layers using the same `nn` module, `Conv2d`. This is the two-dimensional convolution operation. Let's examine one of them and then dive into the impelmentation of the operation:

In [2]:
conv1 = nn.Conv2d(1, 20, 5, 1)

Our `Net` class inherits from `nn.Module`. Each `Module` is defined in `nn.modules`. The base class `Module` is defined in `module.py` and inherits from something called `object`.

`Conv2d` is a class within `conv.py`. If we search for `Conv2d` within that file, we see that it inherits from `_ConvNd`, which in turn inherits from `Module`. Under `Args:`, we see that our four arguments correspond to the following:

`in_channels`: Number of input channels. Our input image is one-channel (grayscale).

`out_channels`: Number of output channels. We tell `Conv2d` how many feature maps we want the function to learn.

`kernel_size`: The size of the filter that generates our output feature maps. If we pass an `int`, PyTorch assumes a square kernel. We can also pass a `tuple` for non-square kernels.

`stride`: As with `kernel_size`, we can define an equal horizontal and vertical stride (`int`). If we want horizontal and vertical strides of different lengths, we can pass a `tuple`.

Interestingly, `conv2d_forward`, which executes the convolution in the `Conv2d` class returns `F.conv2d`, the `conv2d` operation as defined in `nn.functional`. Let's turn to `nn.functional` to understand how `conv2d` is actually performed.

If we navigate in `nn` to `functional.py`, we see that we're just calling `_add_docstr(torch.conv2d)` followed by some documentation. The actual convolution isn't performed here. It's somewhere in the C/C++ backend, though it's not clear where.

One way to slice into this is to examine the `native_functions.yaml` file in `pytorch/master/aten/src/ATen/native/`. All (native) PyTorch functions (including `conv2d`) are defined in this file. What do we see if we search for `conv2d` (as of 10/26/19)?

Searching through the file, we see that the `conv2d` associated with `python_module: nn` is still implemented in `legacy::cpu::_thnn`. This is the `THNN` in `pytorch/aten/src`. If we navigate to `THNN/generic`, we see a file `SpatialConvolutionMM.c` for two-dimensional convolutions in C. There's also `aten/src/TH/generic/THTensorConv.cpp`, which defines "regular" and "SEE-based" convolutions in C++. For now, let's just say that convolutions are implemented somewhere in the (apparently legacy) backend). We have other ops to cover.

The other op in `__init__` is `nn.Linear` (e.g. from `fc1`):

In [4]:
fc1 = nn.Linear(4*4*50, 500)

This simply performs a linear transformation that takes 800 inputs and generates 500 outputs. Presumably this will be defined as a module in `nn` which will call something in `nn.functional`, but we want to examine the C/C++ implementation.

We also see `linear` defined in `native_functions.yaml`. If we navigate back to `ATen/native`, we see a `Linear.cpp` file. There's also a `Convolution.cpp` file. This contains an `at::Tensor conv2d` function that returns `at::convolution` with a particular set of arguments. `at::convolution` in turn calls `at::_convolution` that calls a different function depending on input parameters:

`use_cudnn_depthwise` -> `at::cudnn_convolution` <br/>
`use_miopen` -> `at::miopen_depthwise_convolution` <br/>
`else` -> `at::thnn_conv_depthwise2d` <br/>

(etc.)

The one that we're using is `thnn_conv_depthwise2d`. If we search `native_functions.yaml`, we see that this is also associated with `python_module: nn`, though it still isn't clear where it's implemented.

For `linear`, it's a bit clearer. In `Linear.cpp`, we see that `linear` returns `at:addmm` (unless we call `is_mkldnn`, in which in returns `at::mkldnn_linear`. If we search `native_functions.yaml`, we see that `addmm` is defined on the CPU in `_th_addmm`. We find the `THTensor_(addmm)` function in `TH/generic/THTensorMath.cpp` which calls `THTensor_(addmmImpl)`. At the end of the day, it's just a (matrix) multiplication and addition!

Returning to `main.py`, we first see `F.relu` in our `forward` method. To make this work without generating a `NameError`, we need to define `x` as a tensor with the input dimensions for `conv1`. We'll just use `torch.ones` and pass the shape of the image (28x28) as the final two parameters. The first parameter is our batch size (1) and the second parameter is our number of channels (1):

In [9]:
x = torch.ones([1, 1, 28, 28])

Now we can pass our data through `F.relu`:

In [11]:
x = F.relu(conv1(x))

In `native_functions.yaml`, we see that `relu` is dispatched with `CPU: relu`. This means the implementation should be in `aten/src/ATen`. We find `relu` and other activation functions in `native/Activation.cpp`, where `relu` returns `at::threshold(self, 0, 0)`. `threshold` is in turn defined in `native_functions.yaml`.

Let's shift focus from low-level implementation to what the operations actually do. So far we have a convolution (sliding a filter across an input set of feature maps and producing an output set of feature maps), a linear transformation (matrix multiplication and addition), and a ReLU activation (the max of zero and the input).

Next we have a max pooling operation. This takes the maximum value over a specified range:

In [12]:
x = F.max_pool2d(x, 2, 2)

Let's just summarize these along with our remaining operations, since we have new ones to cover in other networks:

`nn.Conv2d`: two-dimensional convolution <br/>
`nn.Linear`: linear transformation <br/>
`F.relu`: ReLU activation <br/>
`F.max_pool2d`: two-dimensional max pooling <br/>
`x.view`: flattening to one dimension <br/>
`F.log_softmax`: log softmax activation <br/>
`optim.SGD`: stochastic gradient descent optimizer

Let's now examine the image classification networks in `pytorch/vision/torchvision/models`. The first network (alphabetically), `alexnet` introduces a couple of new operations:

`nn.AdaptiveAvgPool2d`: two-dimensional average pooling (generates given output size) <br/>
`nn.Dropout`: randomly removes nodes from input tensor with specified probability <br/>

`densenet` only introduces one additional operation, `BatchNorm2d`, but this requires some explanation. Normalization and other layers and functions are documented in https://pytorch.org/docs/stable/nn.html.

`BatchNorm1d` applies batch normalization over a two or three-dimensional input (batch, channel, 1D data) using the following formula:

$y = \frac{x - E[x]}{\sqrt{Var[x] + \epsilon}} * \gamma + \beta$

Here $E[x]$ is our mean, and $\sqrt{Var[x]+\epsilon}$ is our standard deviation (with an epsilon tacked on). We use this mean and standard deviation to normalize our input $x$.

$\gamma$ and $\beta$ are "learnable parameter vectors" of the same size as our input. The values of $\gamma$ are initialized with ones and the values of $\beta$ are initialized with zeroes.

We compute our mean and variance with momentum (default: 0.1).

`BatchNorm2d` and `BatchNorm3d` use this same function but take four and five-dimensional inputs, respectively.

`GroupNorm`, or Group Normalization uses this same function(?) but calculates the mean and standard deviation separately for a number of groups defined in `num_groups`. Each group contains a number of channels equal to the total number of channels divided by the number of groups.

`SyncBatchNorm` generalizes batch normalization(?) to an N-dimensional input, computes mean and standard deviation for each dimension "over all mini-batches of the same process groups."

It appears that the documentation may be incorrect. The batch normalization function keeps getting copied over and over, and for `InstanceNorm1d`, it's a copy of the batch norm equation *and* text?

Other normalization functions include `InstanceNorm2d`, `InstanceNorm3d`, `LayerNorm` (layer normalization), and `LocalResponseNorm` (local response normalization).

`googlenet` constructs a set of layers (e.g. `BasicConv2d`) from operations in `torch.nn`. The interesting thing about `googlenet` is the use of Inception modules, but we're just concerned for now about the basic operations used to construct the network. The only new one here is apparently `torch.flatten`, which seems to do the same thing as the `view` method (reducing inputs to one dimension).

`inception` uses a similar architecture as `googlenet` but no new ops.

It seems that we can construct most convolutional neural networks for image classification just by using convolutional and linear layers with ReLU activations and some other stuff (e.g. batch norm) thrown in there. Things presumably get more interesting when we move on to detection? Let's examine `vision/torchvision/models/detection` in our next notebook.